In [23]:
!wget -O QnA_toefl.csv https://raw.githubusercontent.com/sathannan/DataScienceTraining/master/QnA_toefl.csv

--2019-09-18 07:29:58--  https://raw.githubusercontent.com/sathannan/DataScienceTraining/master/QnA_toefl.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13713 (13K) [text/plain]
Saving to: ‘QnA_toefl.csv’

100%[======================================>] 13,713      --.-K/s   in 0.001s  

2019-09-18 07:29:58 (11.9 MB/s) - ‘QnA_toefl.csv’ saved [13713/13713]



In [24]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/dsxuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
import pandas as pd
data = pd.read_csv("QnA_toefl.csv")

In [26]:
data.head()

,Question,Answer
0,What is the TOEFL test,The TOEFL test measures the ability of non-nat...
1,What is the difference between the TOEFL iBT t...,"The TOEFL iBT test, delivered via the internet..."
2,How do I know what an institution's score requ...,Each institution sets its own score requiremen...
3,How often is the test given,"The TOEFL iBT test is given on fixed dates, mo..."
4,How do I find testing locations and dates,Choose an available test date from among testi...


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    input='content',     # input is actual text
    lowercase=True,      # convert to lower case before tokenizing
    stop_words='english' # remove stop words
)
transformed_data = vectorizer.fit_transform(data['Question'])

In [28]:
user_input = "what is TOEFL test"
transformed_user_input = vectorizer.transform([user_input])

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sims = []
for question in data['Question']:
    sims = cosine_similarity(vectorizer.transform([question]), transformed_user_input)
    cos_sims.append(sims)
ind = sorted(range(len(cos_sims)), key=lambda i: cos_sims[i], reverse=True)[:3]
for index in ind:
    print(data.iloc[index,1])
    print("Score:",cos_sims[index])
    print("\n")

The TOEFL test measures the ability of non-native English speakers to use and understand the English language as it is heard, spoken, read and written in the university classroom.
Score: [[1.]]


It's the most highly respected around the world. It's the most widely accepted. It's the most accurately measured. It's the most fair and unbiased. It's the most convenient.
Score: [[0.68354015]]


Plan to take the TOEFL test 2 to 3 months before your earliest application or other deadline so your scores arrive at your institutions or agencies in time. Find application deadlines by checking each score recipient s website.
Score: [[0.66758818]]




In [30]:
!pip install python-telegram-bot

In [ ]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

def start(bot, update):
  update.message.reply_text("I'm a bot, Nice to meet you!")

def greet(bot, update):
  update.message.reply_text("Good Morning!")
  
def getAnswer(bot, update):
    question = update.message.text
    print(question)
    transformed_user_input = vectorizer.transform([question])
    cos_sims = []
    for question in data['Question']:
        sims = cosine_similarity(vectorizer.transform([question]), transformed_user_input)
        cos_sims.append(sims)
    ind = cos_sims.index(max(cos_sims))   
    update.message.reply_text(data.iloc[ind,1])

def main():
  # Create Updater object and attach dispatcher to it
  updater = Updater("765915741:AAFRlwMDq0p8MUdnTQUeW7mT4lsDVEe7DJY")
  dispatcher = updater.dispatcher
  print("Bot started")

  # Add command handler to dispatcher
  start_handler = CommandHandler('start',start)
  dispatcher.add_handler(start_handler)

  greet_handler = CommandHandler('greet',greet)
  dispatcher.add_handler(greet_handler)
    
  getAnswer_handler = MessageHandler(Filters.text, getAnswer)
  dispatcher.add_handler(getAnswer_handler)

  # Start the bot
  updater.start_polling()

  # Run the bot until you press Ctrl-C
  updater.idle()

if __name__ == '__main__':
  main()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:22: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details


Bot started
